In [ ]:
%reset
import csv
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.ndimage import median_filter
import numpy as np
import pandas as pd
from scipy import signal
import statistics as stats
import matplotlib.dates as mdates
from matplotlib.transforms import Transform
from matplotlib.ticker import (
    AutoLocator, AutoMinorLocator)
from scipy import stats
import scipy.sparse as sparse

In [ ]:
def weighted_average(distribution, weights):
    return round(sum([distribution[i]*weights[i] for i in range(len(distribution))])/sum(weights),2)

def import_csv(csvfilename):
    data = []
    with open(csvfilename, "r", encoding="utf-8", errors="ignore") as scraped:
        reader = csv.reader(scraped, delimiter=' ', skipinitialspace='True')
        next(reader)
        row_index = 0
        for row in reader:
            if row:  # avoid blank lines
                #row_index += 1
                #columns = [str(row_index), row[0], row[1], row[2], row[3]]
                columns = [row[0], row[1]]
                data.append(columns)

    return data

def import_csv2(csvfilename):
    data = []
    with open(csvfilename, "r", encoding="utf-8", errors="ignore") as scraped:
        reader = csv.reader(scraped, delimiter=' ', skipinitialspace='True')
        next(reader)
        row_index = 0
        for row in reader:
            if row:  # avoid blank lines
                #row_index += 1
                #columns = [str(row_index), row[0], row[1], row[2], row[3]]
                columns = [row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8],
                           row[9], row[10] ]
                data.append(columns)

    return data

def import_csv3(csvfilename):
    data = []
    with open(csvfilename, "r", encoding="utf-8", errors="ignore") as scraped:
        reader = csv.reader(scraped, delimiter=' ', skipinitialspace='True')
        next(reader)
        row_index = 0
        for row in reader:
            if row:  # avoid blank lines
                #row_index += 1
                #columns = [str(row_index), row[0], row[1], row[2], row[3]]
                columns = [row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8],
                           row[9], row[10], row[11] ]
                data.append(columns)

    return data

def import_csv4(csvfilename):
    data = []
    with open(csvfilename, "r", encoding="utf-8", errors="ignore") as scraped:
        reader = csv.reader(scraped, delimiter=' ', skipinitialspace='True')
        next(reader)
        row_index = 0
        for row in reader:
            if row:  # avoid blank lines
                #row_index += 1
                #columns = [str(row_index), row[0], row[1], row[2], row[3]]
                columns = [row[0], row[1], row[2], row[3], row[4]]
                data.append(columns)

    return data

In [ ]:
tcp_cca = (['TcpCubic','TcpBbr',
            'TcpQtColFair_1.1minRTT', 'TcpQtColFair_1.2minRTT', 'TcpQtColFair_1.3minRTT', 
            'TcpQtColFair_1.4minRTT', 'TcpQtColFair_1.5minRTT', 'TcpQtCol_1.2minRTT', 'TcpQtCol_1.5minRTT'
           ])
BDP = 172
buffer_size = 1727 
print(buffer_size, tcp_cca)

In [ ]:
num_flows = 5
cwnd = {}
agg_cwnd = {}
inflight = {}
rtt = {}

for algorithm in tcp_cca:
    cwnd[algorithm] = {}
    agg_cwnd[algorithm] = {}
    inflight[algorithm] = {}
    rtt[algorithm] = {}


    agg_cwnd[algorithm] = {'time': list(), 'agg_cwnd': list(), 'agg_inflight': list(),
                'aveRtt': list(), 'agg_insThruput': list(), 'agg_aveThruput': list(), 
                'agg_insGoodput': list(), 'agg_aveGoodput': list(), 'agg_TxPkts': list(), 
                'agg_ReTxPkts': list(), 'fairness': list(), 'agg_cost': list()}
        
    for i in range(num_flows):
        cwnd[algorithm][i] = {'time': list(), 'cwnd': list(), 'inflight': list(),
                'rtt': list(), 'insThruput': list(), 'aveThruput': list(), 
                'insGoodput': list(), 'aveGoodput': list(), 'TxPkts': list(), 'ReTxPkts': list(), 
                'cost': list()}
                                       
        inflight[algorithm][i] = {'time': list(), 'inflight': list(), 'cwnd': list(), 
                                           'rtt': list(), 'cost': list()}
        rtt[algorithm][i] = {'time': list(), 'rtt': list()}

cwnd_files = []
agg_cwnd_files = []
inflight_files = []
rtt_files = []
                                               
for algorithm in tcp_cca:
    print(algorithm)
    
    agg_cwnd_files.append("../results/qt_multiflows/" + str(algorithm) + "/" + 
                str(num_flows)  + "-flows/100Mbps-20ms/" + str(buffer_size) + "p-btlqueue/" + 
                algorithm + "-" + str(num_flows) + "-flows-100Mbps-20ms-" + str(buffer_size) +
                "p-" + "Agg" + "-cwnd.data")
    cwnd_flows_files = []
    inflight_flows_files = []
    rtt_flows_files = []
    
    for i in range(num_flows):
        cwnd_flows_files.append("../results/qt_multiflows/" + str(algorithm) + "/" + 
                    str(num_flows)  + "-flows/100Mbps-20ms/" + str(buffer_size) + "p-btlqueue/" + 
                    algorithm + "-" + str(num_flows) + "-flows-100Mbps-20ms-" + str(buffer_size) +
                    "p-flow" + str(i) + "-cwnd.data")
 #       inflight_flows_files.append("../results/multiflows/" + str(algorithm) + "/" + 
 #                   str(num_flows)  + "-flows/100Mbps-20ms/" + str(buffer_size) + "p-btlqueue/" + 
 #                   algorithm + "-" + str(num_flows) + "-flows-100Mbps-20ms-" + str(buffer_size) +
 #                   "p-flow" + str(i) + "-inflight.data")
 #       rtt_flows_files.append("../results/multiflows/" + str(algorithm) + "/" + 
 #                       str(num_flows)  + "-flows/100Mbps-20ms/" + str(buffer_size) + "p-btlqueue/" + 
 #                       algorithm + "-" + str(num_flows) + "-flows-100Mbps-20ms-" + str(buffer_size) +
 #                       "p-flow" + str(i) + "-rtt.data")
        
    cwnd_files.append(cwnd_flows_files)
    inflight_files.append(inflight_flows_files)
    rtt_files.append(rtt_flows_files)



In [ ]:
print(agg_cwnd_files[0])
print(agg_cwnd_files[1])
print(agg_cwnd_files[2])

In [ ]:

for algorithm  in tcp_cca:
    index_alg = tcp_cca.index(algorithm)
    print(algorithm,index_alg)

    agg_cwnd_data_in = import_csv3(agg_cwnd_files[index_alg])
    for row in agg_cwnd_data_in: 
        agg_cwnd[algorithm]['time'].append({row[0]})
        agg_cwnd[algorithm]['agg_cwnd'].append({row[1]})
        agg_cwnd[algorithm]['agg_inflight'].append({row[2]})
        agg_cwnd[algorithm]['aveRtt'].append({row[3]})
        agg_cwnd[algorithm]['agg_insThruput'].append({row[4]})
        agg_cwnd[algorithm]['agg_aveThruput'].append({row[5]})
        agg_cwnd[algorithm]['agg_insGoodput'].append({row[6]})
        agg_cwnd[algorithm]['agg_aveGoodput'].append({row[7]})
        agg_cwnd[algorithm]['agg_TxPkts'].append({row[8]})
        agg_cwnd[algorithm]['agg_ReTxPkts'].append({row[9]})
        agg_cwnd[algorithm]['fairness'].append({row[10]})
        agg_cwnd[algorithm]['agg_cost'].append({row[11]})

    for i in range(num_flows):
        cwnd_data_in = import_csv2(cwnd_files[index_alg][i])
        #inflight_data_in = import_csv4(inflight_files[index_alg][i])
        #rtt_data_in = import_csv(rtt_files[index_alg][i])

        for row in cwnd_data_in:
            cwnd[algorithm][i]['time'].append({row[0]})
            cwnd[algorithm][i]['cwnd'].append({row[1]})
            cwnd[algorithm][i]['inflight'].append({row[2]})
            cwnd[algorithm][i]['rtt'].append({row[3]})
            cwnd[algorithm][i]['insThruput'].append({row[4]})
            cwnd[algorithm][i]['aveThruput'].append({row[5]})
            cwnd[algorithm][i]['insGoodput'].append({row[6]})
            cwnd[algorithm][i]['aveGoodput'].append({row[7]})
            cwnd[algorithm][i]['TxPkts'].append({row[8]})
            cwnd[algorithm][i]['ReTxPkts'].append({row[9]})
            cwnd[algorithm][i]['cost'].append({row[10]})


In [ ]:
cwnd_time = []
cwnd_value = []
inflight_value = []
rtt_value = []
insThruput_value = []
aveThruput_value = []
insGoodput_value = []
aveGoodput_value = []
cost_value = []

inflight2_time = []
inflight2_value = []
cwnd2_value = []
rtt2_value = []
cost2_value = []
rtt3_time = []
rtt3_value = []

agg_cwnd_time = []
agg_cwnd_value = []
agg_inflight_value = []
agg_rtt_value = []
agg_insThruput_value = []
agg_aveThruput_value = []
agg_insGoodput_value = []
agg_aveGoodput_value = []
fairness_value = []
agg_cost_value = []

for algorithm in tcp_cca:
    print(algorithm)
    cwnd_time_ = []
    cwnd_value_ = []
    inflight_value_ = []
    rtt_value_ = []
    insThruput_value_ = []
    aveThruput_value_ = []
    insGoodput_value_ = []
    aveGoodput_value_ = []
    cost_value_ = []
    

    agg_cwnd_time.append(pd.DataFrame(agg_cwnd[algorithm]['time']))
    agg_cwnd_value.append(pd.DataFrame(agg_cwnd[algorithm]['agg_cwnd']))
    agg_inflight_value.append(pd.DataFrame(agg_cwnd[algorithm]['agg_inflight']))
    agg_rtt_value.append(pd.DataFrame(agg_cwnd[algorithm]['aveRtt']))
    agg_insThruput_value.append(pd.DataFrame(agg_cwnd[algorithm]['agg_insThruput']))  
    agg_aveThruput_value.append(pd.DataFrame(agg_cwnd[algorithm]['agg_aveThruput']))  
    agg_insGoodput_value.append(pd.DataFrame(agg_cwnd[algorithm]['agg_insGoodput']))  
    agg_aveGoodput_value.append(pd.DataFrame(agg_cwnd[algorithm]['agg_aveGoodput'])) 
    fairness_value.append(pd.DataFrame(agg_cwnd[algorithm]['fairness'])) 
    agg_cost_value.append(pd.DataFrame(agg_cwnd[algorithm]['agg_cost'])) 
    
    
    for i in range(num_flows):
        cwnd_time_.append(pd.DataFrame(cwnd[algorithm][i]['time']))
        cwnd_value_.append(pd.DataFrame(cwnd[algorithm][i]['cwnd']))
        cost_value_.append(pd.DataFrame(cwnd[algorithm][i]['cost']))
        inflight_value_.append(pd.DataFrame(cwnd[algorithm][i]['inflight']))
        rtt_value_.append(pd.DataFrame(cwnd[algorithm][i]['rtt']))
        insThruput_value_.append(pd.DataFrame(cwnd[algorithm][i]['insThruput']))  
        aveThruput_value_.append(pd.DataFrame(cwnd[algorithm][i]['aveThruput']))  
        insGoodput_value_.append(pd.DataFrame(cwnd[algorithm][i]['insGoodput']))  
        aveGoodput_value_.append(pd.DataFrame(cwnd[algorithm][i]['aveGoodput'])) 
        
    cwnd_time.append(cwnd_time_)
    cwnd_value.append(cwnd_value_)
    cost_value.append(cost_value_)
    inflight_value.append(inflight_value_)
    rtt_value.append(rtt_value_)
    insThruput_value.append(insThruput_value_)
    aveThruput_value.append(aveThruput_value_)
    insGoodput_value.append(insGoodput_value_)
    aveGoodput_value.append(aveGoodput_value_)


In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

ax1.plot(
        agg_cwnd_time[0].to_numpy(dtype=float), agg_cwnd_value[0].to_numpy(dtype=float)/1448.0, '.-',
        agg_cwnd_time[1].to_numpy(dtype=float), agg_cwnd_value[1].to_numpy(dtype=float)/1448,'.-',
        agg_cwnd_time[2].to_numpy(dtype=float), agg_cwnd_value[2].to_numpy(dtype=float)/1448,'.-',
        agg_cwnd_time[3].to_numpy(dtype=float), agg_cwnd_value[3].to_numpy(dtype=float)/1448,'.-',
        agg_cwnd_time[4].to_numpy(dtype=float), agg_cwnd_value[4].to_numpy(dtype=float)/1448,'.-',
        agg_cwnd_time[5].to_numpy(dtype=float), agg_cwnd_value[5].to_numpy(dtype=float)/1448,'.-',
        )
#ax1.step(
#        agg_cwnd_time[1].to_numpy(dtype=float), agg_cwnd_value[1].to_numpy(dtype=float),'x-', where='post'
#        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
plt.ylabel(r'Cwnd (in Packets)')
#plt.xscale('log')
#plt.xticks(x,values)
#plt.yscale('log')
#plt.ylim(0,800)
plt.xlim(0,100)
plt.grid(True)
plt.legend([
            tcp_cca[0].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[1].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[2].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[3].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[4].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[5].replace('minRTT','').replace('Tcp','')
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
plt.savefig('multiple_flows/5_flows_agg_cwnd.png', transparent=True)


In [ ]:
ig, ax1 = plt.subplots(constrained_layout=True)

ax1.plot(
        agg_cwnd_time[0].to_numpy(dtype=float), agg_inflight_value[0].to_numpy(dtype=float)/1448.0, '.-',
        agg_cwnd_time[1].to_numpy(dtype=float), agg_inflight_value[1].to_numpy(dtype=float)/1448,'.-',
        )
#ax1.step(
#        agg_cwnd_time[1].to_numpy(dtype=float), agg_inflight_value[1].to_numpy(dtype=float),'x-', where='post'
#        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
plt.ylabel(r'InFlight (in Packets)')
#plt.xscale('log')
#plt.xticks(x,values)
#plt.yscale('log')
#plt.ylim(0,800)
plt.xlim(0,40)
plt.grid(True)
plt.legend([
            tcp_cca[0].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[1].replace('minRTT','').replace('Tcp','')
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
plt.savefig('multiple_flows/5_flows_agg_inflight_bbr_cubic.png', transparent=True)

In [ ]:
ig, ax1 = plt.subplots(constrained_layout=True)

ax1.plot(
        agg_cwnd_time[2].to_numpy(dtype=float), agg_inflight_value[2].to_numpy(dtype=float)/1448,'.-',
        agg_cwnd_time[3].to_numpy(dtype=float), agg_inflight_value[3].to_numpy(dtype=float)/1448,'.-',
        )
#ax1.step(
#        agg_cwnd_time[1].to_numpy(dtype=float), agg_inflight_value[1].to_numpy(dtype=float),'x-', where='post'
#        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
plt.ylabel(r'InFlight (in Packets)')
#plt.xscale('log')
#plt.xticks(x,values)
#plt.yscale('log')
#plt.ylim(0,800)
plt.xlim(0,50)
plt.grid(True)
plt.legend([
            tcp_cca[2].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[3].replace('minRTT','').replace('Tcp','')
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
plt.savefig('multiple_flows/5_flows_agg_inflight_qtcol.png', transparent=True)

In [ ]:
ig, ax1 = plt.subplots(constrained_layout=True)

ax1.plot(
        agg_cwnd_time[1].to_numpy(dtype=float), agg_inflight_value[1].to_numpy(dtype=float)/1448,'.-',
        agg_cwnd_time[2].to_numpy(dtype=float), agg_inflight_value[2].to_numpy(dtype=float)/1448,'.-',
        agg_cwnd_time[3].to_numpy(dtype=float), agg_inflight_value[3].to_numpy(dtype=float)/1448,'.-',
        agg_cwnd_time[4].to_numpy(dtype=float), agg_inflight_value[4].to_numpy(dtype=float)/1448,'.-',
        agg_cwnd_time[5].to_numpy(dtype=float), agg_inflight_value[5].to_numpy(dtype=float)/1448,'.-',
        agg_cwnd_time[6].to_numpy(dtype=float), agg_inflight_value[6].to_numpy(dtype=float)/1448,'.-',
        )
#ax1.step(
#        agg_cwnd_time[1].to_numpy(dtype=float), agg_inflight_value[1].to_numpy(dtype=float),'x-', where='post'
#        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
plt.ylabel(r'InFlight (in Packets)')
#plt.xscale('log')
#plt.xticks(x,values)
#plt.yscale('log')
#plt.ylim(0,800)
plt.xlim(0,100)
plt.grid(True)
plt.legend([
            tcp_cca[1].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[2].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[3].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[4].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[5].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[6].replace('minRTT','').replace('Tcp','')
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
plt.savefig('multiple_flows/5_flows_agg_inflight__best_delay_based.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

ax1.plot(
        agg_cwnd_time[0].to_numpy(dtype=float), agg_rtt_value[0].to_numpy(dtype=float), '.-',
        agg_cwnd_time[1].to_numpy(dtype=float), agg_rtt_value[1].to_numpy(dtype=float),'.-',
        )
#ax1.step(
#        agg_cwnd_time[1].to_numpy(dtype=float), agg_rtt_value[1].to_numpy(dtype=float),'x-', where='post'
#        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
plt.ylabel(r'rtt (in Packets)')
#plt.xscale('log')
#plt.xticks(x,values)
#plt.yscale('log')
#plt.ylim(0,800)
plt.xlim(0,40)
plt.grid(True)
plt.legend([
            tcp_cca[0].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[1].replace('minRTT','').replace('Tcp','')
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
plt.savefig('multiple_flows/5_flows_avett_bbr_cubic.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

ax1.plot(
        agg_cwnd_time[2].to_numpy(dtype=float), agg_rtt_value[2].to_numpy(dtype=float),'.-',
        agg_cwnd_time[3].to_numpy(dtype=float), agg_rtt_value[3].to_numpy(dtype=float),'.-',
        )
#ax1.step(
#        agg_cwnd_time[1].to_numpy(dtype=float), agg_rtt_value[1].to_numpy(dtype=float),'x-', where='post'
#        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
plt.ylabel(r'rtt (in Packets)')
#plt.xscale('log')
#plt.xticks(x,values)
#plt.yscale('log')
#plt.ylim(0,800)
plt.xlim(0,50)
plt.grid(True)
plt.legend([
            tcp_cca[2].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[3].replace('minRTT','').replace('Tcp','')
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
plt.savefig('multiple_flows/5_flows_avertt_qtcol.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

ax1.plot(
        agg_cwnd_time[2].to_numpy(dtype=float), agg_rtt_value[2].to_numpy(dtype=float),',-',
        agg_cwnd_time[6].to_numpy(dtype=float), agg_rtt_value[6].to_numpy(dtype=float),',-',
        )
#ax1.step(
#        agg_cwnd_time[1].to_numpy(dtype=float), agg_rtt_value[1].to_numpy(dtype=float),'x-', where='post'
#        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
plt.ylabel(r'rtt (in Packets)')
#plt.xscale('log')
#plt.xticks(x,values)
#plt.yscale('log')
#plt.ylim(0,800)
plt.xlim(0,200)
plt.grid(True)
plt.legend([
            tcp_cca[2].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[6].replace('minRTT','').replace('Tcp','')
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
plt.savefig('multiple_flows/5_flows_avertt_tcpqtcolfair.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

ax1.plot(
        agg_cwnd_time[1].to_numpy(dtype=float), agg_rtt_value[1].to_numpy(dtype=float),'+',
        agg_cwnd_time[2].to_numpy(dtype=float), agg_rtt_value[2].to_numpy(dtype=float),'+',
        agg_cwnd_time[3].to_numpy(dtype=float), agg_rtt_value[3].to_numpy(dtype=float),'+',
        )
#ax1.step(
#        agg_cwnd_time[1].to_numpy(dtype=float), agg_rtt_value[1].to_numpy(dtype=float),'x-', where='post'
#        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
plt.ylabel(r'rtt (in Packets)')
#plt.xscale('log')
#plt.xticks(x,values)
#plt.yscale('log')
#plt.ylim(0,800)
plt.xlim(0,100)
plt.grid(True)
plt.legend([
            tcp_cca[1].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[2].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[3].replace('minRTT','').replace('Tcp','')
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
plt.savefig('multiple_flows/5_flows_avertt_best_delay_based.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

ax1.plot(
        agg_cwnd_time[5].to_numpy(dtype=float), agg_rtt_value[5].to_numpy(dtype=float),'C3,-',
        agg_cwnd_time[0].to_numpy(dtype=float), agg_rtt_value[0].to_numpy(dtype=float),'C2,-'
        )
#ax1.step(
#        agg_cwnd_time[1].to_numpy(dtype=float), agg_rtt_value[1].to_numpy(dtype=float),'x-', where='post'
#        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
plt.ylabel(r'rtt (in Packets)')
#plt.xscale('log')
#plt.xticks(x,values)
#plt.yscale('log')
#plt.ylim(0,800)
plt.xlim(0,200)
plt.grid(True)
plt.legend([
            tcp_cca[5].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[0].replace('minRTT','').replace('Tcp','')
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
plt.savefig('multiple_flows/5_flows_avertt_worst_delay_based.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

ax1.plot(
        agg_cwnd_time[0].to_numpy(dtype=float), fairness_value[0].to_numpy(dtype=float), '-',
        agg_cwnd_time[1].to_numpy(dtype=float), fairness_value[1].to_numpy(dtype=float),'-',
        agg_cwnd_time[2].to_numpy(dtype=float), fairness_value[2].to_numpy(dtype=float),'-',
        agg_cwnd_time[3].to_numpy(dtype=float), fairness_value[3].to_numpy(dtype=float),'-',
        agg_cwnd_time[6].to_numpy(dtype=float), fairness_value[6].to_numpy(dtype=float),'-',
        agg_cwnd_time[7].to_numpy(dtype=float), fairness_value[7].to_numpy(dtype=float),'-',
        agg_cwnd_time[8].to_numpy(dtype=float), fairness_value[8].to_numpy(dtype=float),'-',
        )
#ax1.step(
#        agg_cwnd_time[1].to_numpy(dtype=float), fairness_value[1].to_numpy(dtype=float),'x-', where='post'
#        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
plt.ylabel(r'Fairness')
#plt.xscale('log')
#plt.xticks(x,values)
#plt.yscale('log')
#plt.ylim(0.2,1.02)
#plt.xlim(0,400)
plt.grid(True)
plt.legend([
            tcp_cca[0], 
            tcp_cca[1].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[2].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[3].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[6].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[7].replace('minRTT','').replace('Tcp',''),
            tcp_cca[8].replace('minRTT','').replace('Tcp',''),
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
plt.savefig('multiple_flows/5_flows_fairness.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

ax1.plot(
        agg_cwnd_time[0].to_numpy(dtype=float), fairness_value[0].to_numpy(dtype=float), '.-',
        agg_cwnd_time[1].to_numpy(dtype=float), fairness_value[1].to_numpy(dtype=float),'.-',
        )
#ax1.step(
#        agg_cwnd_time[1].to_numpy(dtype=float), fairness_value[1].to_numpy(dtype=float),'x-', where='post'
#        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
plt.ylabel(r'Fairness')
#plt.xscale('log')
#plt.xticks(x,values)
#plt.yscale('log')
plt.ylim(0.2,1.02)
#plt.xlim(0,400)
plt.grid(True)
plt.legend([
            tcp_cca[0].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[1].replace('minRTT','').replace('Tcp','')
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
#plt.savefig('multiple_flows/multiple_flows_fairness.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

ax1.plot(
        agg_cwnd_time[2].to_numpy(dtype=float), fairness_value[2].to_numpy(dtype=float),'-',
        agg_cwnd_time[3].to_numpy(dtype=float), fairness_value[3].to_numpy(dtype=float),'-',
        agg_cwnd_time[4].to_numpy(dtype=float), fairness_value[4].to_numpy(dtype=float),'-',
        agg_cwnd_time[5].to_numpy(dtype=float), fairness_value[5].to_numpy(dtype=float),'-',
        agg_cwnd_time[6].to_numpy(dtype=float), fairness_value[6].to_numpy(dtype=float),'-',
        agg_cwnd_time[7].to_numpy(dtype=float), fairness_value[7].to_numpy(dtype=float),'-',
        agg_cwnd_time[8].to_numpy(dtype=float), fairness_value[8].to_numpy(dtype=float),'-'
        )
#ax1.step(
#        agg_cwnd_time[1].to_numpy(dtype=float), fairness_value[1].to_numpy(dtype=float),'x-', where='post'
#        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
plt.ylabel(r'Fairness')
#plt.xscale('log')
#plt.xticks(x,values)
#plt.yscale('log')
plt.ylim(0.2,1.02)
#plt.xlim(0,400)
plt.grid(True)
plt.legend([
            tcp_cca[2].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[3].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[4].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[5].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[6].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[7].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[8].replace('minRTT','').replace('Tcp','')
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
plt.savefig('multiple_flows/5_flows_fairness_tcpqt.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

ax1.plot(
        agg_cwnd_time[0].to_numpy(dtype=float), fairness_value[0].to_numpy(dtype=float), '.-',
        agg_cwnd_time[1].to_numpy(dtype=float), fairness_value[1].to_numpy(dtype=float),'.-',
        )
#ax1.step(
#        agg_cwnd_time[1].to_numpy(dtype=float), fairness_value[1].to_numpy(dtype=float),'x-', where='post'
#        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
plt.ylabel(r'Fairness')
#plt.xscale('log')
#plt.xticks(x,values)
#plt.yscale('log')
plt.ylim(0.2,1.02)
#plt.xlim(0,400)
plt.grid(True)
plt.legend([
            tcp_cca[0].replace('minRTT','').replace('Tcp',''), tcp_cca[1].replace('minRTT','').replace('Tcp','')
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
plt.savefig('multiple_flows/5_flows_fairness_cubic_bbr.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

ax1.plot(
        agg_cwnd_time[0].to_numpy(dtype=float), agg_aveThruput_value[0].to_numpy(dtype=float), '_-',
        agg_cwnd_time[1].to_numpy(dtype=float), agg_aveThruput_value[1].to_numpy(dtype=float),'_-',
        agg_cwnd_time[2].to_numpy(dtype=float), agg_aveThruput_value[2].to_numpy(dtype=float),'_-',
        agg_cwnd_time[3].to_numpy(dtype=float), agg_aveThruput_value[3].to_numpy(dtype=float),'_-',
        agg_cwnd_time[5].to_numpy(dtype=float), agg_aveThruput_value[5].to_numpy(dtype=float),'_-',
        agg_cwnd_time[8].to_numpy(dtype=float), agg_aveThruput_value[8].to_numpy(dtype=float),'_-',
        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
plt.ylabel(r'Aggregate thruput')
plt.ylim(90,97)
#plt.xlim(0,200)
plt.grid(True)
plt.legend([
            tcp_cca[0].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[1].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[2].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[3].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[5].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[8].replace('minRTT','').replace('Tcp','')
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
plt.savefig('multiple_flows/5_flows_thruput.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)
ax2 = ax1.twinx()

ax1.plot(
        agg_cwnd_time[0].to_numpy(dtype=float), agg_aveGoodput_value[0].to_numpy(dtype=float), '-',
        agg_cwnd_time[1].to_numpy(dtype=float), agg_aveGoodput_value[1].to_numpy(dtype=float),'-',
        agg_cwnd_time[2].to_numpy(dtype=float), agg_aveGoodput_value[2].to_numpy(dtype=float),'-',
        agg_cwnd_time[3].to_numpy(dtype=float), agg_aveGoodput_value[3].to_numpy(dtype=float),'-',
        agg_cwnd_time[6].to_numpy(dtype=float), agg_aveGoodput_value[6].to_numpy(dtype=float),'-',
        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
#plt.ylabel(r'Goodput (Mbps)')
ax2.set_ylabel(r'Effective utilisation')
ax1.set_ylabel(r'Goodput (Mbps)')
#plt.xscale('log')
#plt.xticks(x,values)
#plt.yscale('log')
#plt.ylim(90,96.2)
ax2.set_ylim(0.90,0.962)
ax1.set_ylim(90,96.2)
plt.xlim(0,200)
plt.grid(True)
ax1.legend([
            tcp_cca[0].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[1].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[2].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[3].replace('minRTT','').replace('Tcp',''), 
            tcp_cca[6].replace('minRTT','').replace('Tcp',''), 
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
plt.savefig('multiple_flows/5_flows_goodput.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

ax1.plot(
        cwnd_time[0][0].to_numpy(dtype=float), aveGoodput_value[0][0].to_numpy(dtype=float), '-',
        cwnd_time[0][1].to_numpy(dtype=float), aveGoodput_value[0][1].to_numpy(dtype=float),'-',
        cwnd_time[0][2].to_numpy(dtype=float), aveGoodput_value[0][2].to_numpy(dtype=float),'-',
        cwnd_time[0][3].to_numpy(dtype=float), aveGoodput_value[0][3].to_numpy(dtype=float),'-',
        cwnd_time[0][4].to_numpy(dtype=float), aveGoodput_value[0][4].to_numpy(dtype=float),'-',
        )
#ax1.step(
#        cwnd_time[0].to_numpy(dtype=float), aveGoodput_value[0].to_numpy(dtype=float),'x-', where='post'
#        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
plt.ylabel(r'Goodput for individual flows (Mbps)')
#plt.xscale('log')
#plt.xticks(x,values)
#plt.yscale('log')
#plt.ylim(6,10)
#plt.xlim(0,200)
plt.grid(True)
plt.legend([
            tcp_cca[0].replace('minRTT','').replace('Tcp','') + " flow1", 
            tcp_cca[0].replace('minRTT','').replace('Tcp','') + " flow2", 
            tcp_cca[0].replace('minRTT','').replace('Tcp','') + " flow3", 
            tcp_cca[0].replace('minRTT','').replace('Tcp','') + " flow4", 
            tcp_cca[0].replace('minRTT','').replace('Tcp','') + " flow5"
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
plt.savefig('multiple_flows/5_flows_goodput_cubic.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

ax1.plot(
        cwnd_time[1][0].to_numpy(dtype=float), aveGoodput_value[1][0].to_numpy(dtype=float), '-',
        cwnd_time[1][1].to_numpy(dtype=float), aveGoodput_value[1][1].to_numpy(dtype=float),'-',
        cwnd_time[1][2].to_numpy(dtype=float), aveGoodput_value[1][2].to_numpy(dtype=float),'-',
        cwnd_time[1][3].to_numpy(dtype=float), aveGoodput_value[1][3].to_numpy(dtype=float),'-',
        cwnd_time[1][4].to_numpy(dtype=float), aveGoodput_value[1][4].to_numpy(dtype=float),'-',
        )
#ax1.step(
#        cwnd_time[1].to_numpy(dtype=float), aveGoodput_value[1].to_numpy(dtype=float),'x-', where='post'
#        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
plt.ylabel(r'Goodput for individual flows (Mbps)')
#plt.xscale('log')
#plt.xticks(x,values)
#plt.yscale('log')
#plt.ylim(6,10)
#plt.xlim(0,200)
plt.grid(True)
plt.legend([
            tcp_cca[1].replace('minRTT','').replace('Tcp','') + " flow1", 
            tcp_cca[1].replace('minRTT','').replace('Tcp','') + " flow2", 
            tcp_cca[1].replace('minRTT','').replace('Tcp','') + " flow3", 
            tcp_cca[1].replace('minRTT','').replace('Tcp','') + " flow4", 
            tcp_cca[1].replace('minRTT','').replace('Tcp','') + " flow5"
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
plt.savefig('multiple_flows/5_flows_goodput_bbr.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

ax1.plot(
        cwnd_time[2][0].to_numpy(dtype=float), aveGoodput_value[2][0].to_numpy(dtype=float), '-',
        cwnd_time[2][1].to_numpy(dtype=float), aveGoodput_value[2][1].to_numpy(dtype=float),'-',
        cwnd_time[2][2].to_numpy(dtype=float), aveGoodput_value[2][2].to_numpy(dtype=float),'-',
        cwnd_time[2][3].to_numpy(dtype=float), aveGoodput_value[2][3].to_numpy(dtype=float),'-',
        cwnd_time[2][4].to_numpy(dtype=float), aveGoodput_value[2][4].to_numpy(dtype=float),'-',
        )
#ax1.step(
#        cwnd_time[1].to_numpy(dtype=float), aveGoodput_value[1].to_numpy(dtype=float),'x-', where='post'
#        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
plt.ylabel(r'Goodput for individual flows (Mbps)')
#plt.xscale('log')
#plt.xticks(x,values)
#plt.yscale('log')
#plt.ylim(6,10)
#plt.xlim(0,200)
plt.grid(True)
plt.legend([
            tcp_cca[2].replace('minRTT','').replace('Tcp','') + " flow1", 
            tcp_cca[2].replace('minRTT','').replace('Tcp','') + " flow2", 
            tcp_cca[2].replace('minRTT','').replace('Tcp','') + " flow3", 
            tcp_cca[2].replace('minRTT','').replace('Tcp','') + " flow4", 
            tcp_cca[2].replace('minRTT','').replace('Tcp','') + " flow5"
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
plt.savefig('multiple_flows/5_flows_goodput_qtcolfair1.1.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

ax1.plot(
        cwnd_time[3][0].to_numpy(dtype=float), aveGoodput_value[3][0].to_numpy(dtype=float), '-',
        cwnd_time[3][1].to_numpy(dtype=float), aveGoodput_value[3][1].to_numpy(dtype=float),'-',
        cwnd_time[3][2].to_numpy(dtype=float), aveGoodput_value[3][2].to_numpy(dtype=float),'-',
        cwnd_time[3][3].to_numpy(dtype=float), aveGoodput_value[3][3].to_numpy(dtype=float),'-',
        cwnd_time[3][4].to_numpy(dtype=float), aveGoodput_value[3][4].to_numpy(dtype=float),'-',
        )
#ax1.step(
#        cwnd_time[1].to_numpy(dtype=float), aveGoodput_value[1].to_numpy(dtype=float),'x-', where='post'
#        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
plt.ylabel(r'Goodput for individual flows (Mbps)')
#plt.xscale('log')
#plt.xticks(x,values)
#plt.yscale('log')
#plt.ylim(6,10)
#plt.xlim(0,200)
plt.grid(True)
plt.legend([
            tcp_cca[3].replace('minRTT','').replace('Tcp','') + " flow1", 
            tcp_cca[3].replace('minRTT','').replace('Tcp','') + " flow2", 
            tcp_cca[3].replace('minRTT','').replace('Tcp','') + " flow3", 
            tcp_cca[3].replace('minRTT','').replace('Tcp','') + " flow4", 
            tcp_cca[3].replace('minRTT','').replace('Tcp','') + " flow5"
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
plt.savefig('multiple_flows/5_flows_goodput_qtcolfair1.2.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

ax1.plot(
        cwnd_time[5][0].to_numpy(dtype=float), aveGoodput_value[5][0].to_numpy(dtype=float), '-',
        cwnd_time[5][1].to_numpy(dtype=float), aveGoodput_value[5][1].to_numpy(dtype=float),',-',
        cwnd_time[5][2].to_numpy(dtype=float), aveGoodput_value[5][2].to_numpy(dtype=float),',-',
        cwnd_time[5][3].to_numpy(dtype=float), aveGoodput_value[5][3].to_numpy(dtype=float),',-',
        cwnd_time[5][4].to_numpy(dtype=float), aveGoodput_value[5][4].to_numpy(dtype=float),',-',
        )
#ax1.step(
#        cwnd_time[1].to_numpy(dtype=float), aveGoodput_value[1].to_numpy(dtype=float),'x-', where='post'
#        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
plt.ylabel(r'Goodput for individual flows (Mbps)')
#plt.xscale('log')
#plt.xticks(x,values)
#plt.yscale('log')
#plt.ylim(6,10)
#plt.xlim(0,200)
plt.grid(True)
plt.legend([
            tcp_cca[5].replace('minRTT','').replace('Tcp','') + " flow1", 
            tcp_cca[5].replace('minRTT','').replace('Tcp','') + " flow2", 
            tcp_cca[5].replace('minRTT','').replace('Tcp','') + " flow3", 
            tcp_cca[5].replace('minRTT','').replace('Tcp','') + " flow4", 
            tcp_cca[5].replace('minRTT','').replace('Tcp','') + " flow5"
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
plt.savefig('multiple_flows/5_flows_goodput_qtcofair1.2.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

ax1.plot(
        cwnd_time[7][0].to_numpy(dtype=float), aveGoodput_value[7][0].to_numpy(dtype=float), ',-',
        cwnd_time[7][1].to_numpy(dtype=float), aveGoodput_value[7][1].to_numpy(dtype=float),',-',
        cwnd_time[7][2].to_numpy(dtype=float), aveGoodput_value[7][2].to_numpy(dtype=float),',-',
        cwnd_time[7][3].to_numpy(dtype=float), aveGoodput_value[7][3].to_numpy(dtype=float),',-',
        cwnd_time[7][4].to_numpy(dtype=float), aveGoodput_value[7][4].to_numpy(dtype=float),',-',
        )
#ax1.step(
#        cwnd_time[1].to_numpy(dtype=float), aveGoodput_value[1].to_numpy(dtype=float),'x-', where='post'
#        )

ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')

plt.xlabel(r'Time (secs)')
plt.ylabel(r'Goodput for individual flows (Mbps)')
#plt.xscale('log')
#plt.xticks(x,values)
#plt.yscale('log')
#plt.ylim(6,10)
#plt.xlim(0,200)
plt.grid(True)
plt.legend([
            tcp_cca[7].replace('minRTT','').replace('Tcp','') + " flow1", 
            tcp_cca[7].replace('minRTT','').replace('Tcp','') + " flow2", 
            tcp_cca[7].replace('minRTT','').replace('Tcp','') + " flow3", 
            tcp_cca[7].replace('minRTT','').replace('Tcp','') + " flow4", 
            tcp_cca[7].replace('minRTT','').replace('Tcp','') + " flow5"
           ], loc = 'best')
plt.show
#tikplt.save("rho_vs_L2_mx1.tex")
plt.savefig('multiple_flows/5_flows_goodput_qtcol1.2.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

x0 = agg_rtt_value[0][0].to_numpy(dtype=float)
x1 = agg_rtt_value[1][0].to_numpy(dtype=float)
x2 = agg_rtt_value[2][0].to_numpy(dtype=float)
x3 = agg_rtt_value[3][0].to_numpy(dtype=float)
x6 = agg_rtt_value[6][0].to_numpy(dtype=float)


plt.boxplot ([x0, x1, x2, x3, x6], 
             labels=[tcp_cca[0].replace('Tcp',''),  
                     tcp_cca[1].replace('minRTT','').replace('Tcp',''), 
                     tcp_cca[2].replace('minRTT','').replace('Tcp',''),
                     tcp_cca[3].replace('minRTT','').replace('Tcp',''), 
                     tcp_cca[6].replace('minRTT','').replace('Tcp',''),
                    ],
            boxprops=dict(color='blue'),
             #whiskerprops=dict(color='red'),
             capprops=dict(color='green'),
             medianprops=dict(color='orange'),
             showfliers=False
             #flierprops=dict(markerfacecolor='red', marker='.')
            )
plt.xticks(rotation = 20)

plt.axhline(y = 0.04, color = 'k', linestyle='--')
plt.axhline(y = 0.080, color = 'k', linestyle='--')

arrowprops = dict(
    arrowstyle="->")
ax1.annotate('baseRTT\n=40ms', xy=(1,0.04), xytext=(1.1,0.001), arrowprops=arrowprops)
ax1.annotate(r'2$\times$'+'\nbaseRTT', xy=(2.5,0.08), xytext=(2.5,0.15), arrowprops=arrowprops)


ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')
plt.ylabel(r'Average RTT (seconds)')

plt.show
plt.savefig('multiple_flows/5_flows_rtt_boxplot_TcpQtColFair.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

x0 = agg_inflight_value[0][0].to_numpy(dtype=float)/1448
x1 = agg_inflight_value[1][0].to_numpy(dtype=float)/1448
x2 = agg_inflight_value[2][0].to_numpy(dtype=float)/1448
x3 = agg_inflight_value[3][0].to_numpy(dtype=float)/1448
x6 = agg_inflight_value[6][0].to_numpy(dtype=float)/1448


plt.boxplot ([x0, x1, x2, x3, x6], 
             labels=[tcp_cca[0].replace('Tcp',''), 
                     tcp_cca[1].replace('Tcp',''),  
                     tcp_cca[2].replace('Tcp','').replace('minRTT',''), 
                     tcp_cca[3].replace('Tcp','').replace('minRTT',''),
                     tcp_cca[6].replace('Tcp','').replace('minRTT',''), 
                    ],
            boxprops=dict(color='blue'),
             #whiskerprops=dict(color='red'),
             capprops=dict(color='green'),
             medianprops=dict(color='orange'),
             showfliers=False,
             #flierprops=dict(color='red',marker='.')
            )
plt.xticks(rotation = 20)

plt.axhline(y = 345, color = 'k', linestyle='--')
plt.axhline(y = 690, color = 'k', linestyle='--')

arrowprops = dict(
    arrowstyle="->")
ax1.annotate(r'BDP=345 packets', xy=(3.5,345), xytext=(3.5,100), arrowprops=arrowprops)
ax1.annotate(r'2$\times$ BDP', xy=(3,690), xytext=(3,1400), arrowprops=arrowprops)

#text_kwargs = dict(ha='center', va='center', fontsize=12, color='C4')
#plt.text(9,2100, 'maximum', **text_kwargs)
#plt.text(9,2100, r'75$^{th}$ %ile', **text_kwargs)


ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')
#plt.xlabel(r'Algorithms')
plt.ylabel(r'In-flight data (packets)')

plt.show
plt.savefig('multiple_flows/5_flows_inflight_boxplot_TcpQtColFair.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

x0 = agg_rtt_value[0][0].to_numpy(dtype=float)
x1 = agg_rtt_value[1][0].to_numpy(dtype=float)
x2 = agg_rtt_value[2][0].to_numpy(dtype=float)
x3 = agg_rtt_value[3][0].to_numpy(dtype=float)
x4 = agg_rtt_value[4][0].to_numpy(dtype=float)
x5 = agg_rtt_value[5][0].to_numpy(dtype=float)
x6 = agg_rtt_value[6][0].to_numpy(dtype=float)
x7 = agg_rtt_value[7][0].to_numpy(dtype=float)
x8 = agg_rtt_value[8][0].to_numpy(dtype=float)

plt.boxplot ([x0, x1, x2, x3, x4, x5, x6], 
             labels=[tcp_cca[0].replace('Tcp',''), tcp_cca[1].replace('Tcp',''), 
                     tcp_cca[2].replace('minRTT','').replace('Tcp',''), 
                     tcp_cca[3].replace('minRTT','').replace('Tcp',''), 
                     tcp_cca[4].replace('minRTT','').replace('Tcp',''), 
                     tcp_cca[5].replace('minRTT','').replace('Tcp',''),
                     tcp_cca[6].replace('minRTT','').replace('Tcp',''), 
                     #tcp_cca[7].replace('minRTT','').replace('Tcp',''),
                     #tcp_cca[8].replace('minRTT','').replace('Tcp','')
                    ],
            boxprops=dict(color='blue'),
             #whiskerprops=dict(color='red'),
             capprops=dict(color='green'),
             medianprops=dict(color='orange'),
             showfliers=False
             #flierprops=dict(markerfacecolor='red', marker='.')
            )
plt.xticks(rotation = 20)

plt.axhline(y = 0.04, color = 'k', linestyle='--')
plt.axhline(y = 0.080, color = 'k', linestyle='--')

arrowprops = dict(
    arrowstyle="->")
ax1.annotate('baseRTT\n=40ms', xy=(1.5,0.04), xytext=(1.5,0.001), arrowprops=arrowprops)
ax1.annotate(r'2$\times$'+'\nbaseRTT', xy=(1.5,0.08), xytext=(1.5,0.15), arrowprops=arrowprops)


ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')
plt.ylabel(r'Average RTT (seconds)')

plt.show
plt.savefig('multiple_flows/5_flows_rtt_boxplot_all.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

x0 = agg_inflight_value[0][0].to_numpy(dtype=float)/1448
x1 = agg_inflight_value[1][0].to_numpy(dtype=float)/1448
x2 = agg_inflight_value[2][0].to_numpy(dtype=float)/1448
x3 = agg_inflight_value[3][0].to_numpy(dtype=float)/1448
x4 = agg_inflight_value[4][0].to_numpy(dtype=float)/1448
x5 = agg_inflight_value[5][0].to_numpy(dtype=float)/1448
x6 = agg_inflight_value[6][0].to_numpy(dtype=float)/1448
x7 = agg_inflight_value[7][0].to_numpy(dtype=float)/1448
x8 = agg_inflight_value[8][0].to_numpy(dtype=float)/1448

plt.boxplot ([x0, x1, x2, x3, x4, x5, x6], 
             labels=[tcp_cca[0].replace('Tcp',''), tcp_cca[1].replace('Tcp',''), 
                     tcp_cca[2].replace('Tcp','').replace('minRTT',''), 
                     tcp_cca[3].replace('Tcp','').replace('minRTT',''), 
                     tcp_cca[4].replace('Tcp','').replace('minRTT',''), 
                     tcp_cca[5].replace('Tcp','').replace('minRTT',''),
                     tcp_cca[6].replace('Tcp','').replace('minRTT',''), 
                    ],
            boxprops=dict(color='blue'),
             #whiskerprops=dict(color='red'),
             capprops=dict(color='green'),
             medianprops=dict(color='orange'),
             showfliers=False,
             #flierprops=dict(color='red',marker='.')
            )
plt.xticks(rotation = 20)

plt.axhline(y = 345, color = 'k', linestyle='--')
plt.axhline(y = 690, color = 'k', linestyle='--')

arrowprops = dict(
    arrowstyle="->")
ax1.annotate('BDP\n=345 packets', xy=(3.5,345), xytext=(4.5,10), arrowprops=arrowprops)
ax1.annotate(r'2$\times$ BDP', xy=(3.5,690), xytext=(3.5,1400), arrowprops=arrowprops)

#text_kwargs = dict(ha='center', va='center', fontsize=12, color='C4')
#plt.text(9,2100, 'maximum', **text_kwargs)
#plt.text(9,2100, r'75$^{th}$ %ile', **text_kwargs)


ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')
#plt.xlabel(r'Algorithms')
plt.ylabel(r'In-flight data (packets)')

plt.show
plt.savefig('multiple_flows/5_flows_inflight_boxplot_all.png', transparent=True)

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

x0 = agg_inflight_value[0][0].to_numpy(dtype=float)/1448
x1 = agg_inflight_value[1][0].to_numpy(dtype=float)/1448
x4 = agg_inflight_value[4][0].to_numpy(dtype=float)/1448
x5 = agg_inflight_value[5][0].to_numpy(dtype=float)/1448
x6 = agg_inflight_value[6][0].to_numpy(dtype=float)/1448
x7 = agg_inflight_value[7][0].to_numpy(dtype=float)/1448
x8 = agg_inflight_value[8][0].to_numpy(dtype=float)/1448

plt.boxplot ([x6], 
             labels=[
                     'Label'
                    ],
            boxprops=dict(color='blue'),
             #whiskerprops=dict(color='red'),
             capprops=dict(color='green'),
             medianprops=dict(color='orange'),
             #flierprops=dict(markerfacecolor='red', marker='o')
            )
plt.xticks(rotation = 20)

arrowprops = dict(
    arrowstyle="->")

ax1.annotate('outliers \n(< 0.4%)', xy=(1.01,2200), xytext=(1.2,2250), arrowprops=arrowprops, color='C4')
ax1.annotate('highest value \nwithin nonoutliers \n(Q4=Q3+1.5*IQR)', xy=(0.965,2115), xytext=(0.65,2100), arrowprops=arrowprops, color='C4')
ax1.annotate(r'75$^{th}$ percentile (Q3)', xy=(1.06,1400), xytext=(1.2,1450), arrowprops=arrowprops, color='C4')
ax1.annotate('median (Q2)', xy=(1.06,1100), xytext=(1.2,1150), arrowprops=arrowprops, color='C4')
ax1.annotate('25$^{th}$ percentile (Q1)', xy=(1.06,900), xytext=(1.2,950), arrowprops=arrowprops, color='C4')
ax1.annotate('lowest value \nwithin nonoutliers \n(Q0=Q1-1.5*IQR)', xy=(0.965,160), xytext=(0.65,230), arrowprops=arrowprops, color='C4')
ax1.annotate('outliers \n(< 0.4%)', xy=(1.01,70), xytext=(1.2,20), arrowprops=arrowprops, color='C4')


plt.annotate("", xy=(0.9,900), xytext=(0.9, 1400),
             arrowprops=dict(arrowstyle="<->", connectionstyle="arc3"))
plt.text(0.8, 1150, 'interquartile \nrange (IQR) \n(50%)',
         {'color': 'black', 'fontsize': 10, 'ha': 'center', 'va': 'center'})

plt.annotate("", xy=(0.6,150), xytext=(0.6, 2150),
             arrowprops=dict(arrowstyle="<->", connectionstyle="arc3"))
plt.text(0.58, 1140, r'$\approx$ 99.2%)',
         {'color': 'black', 'fontsize': 10, 'ha': 'center', 'va': 'center'}, rotation=90)

#text_kwargs = dict(ha='center', va='center', fontsize=12, color='C4')
#plt.text(9,2100, 'maximum', **text_kwargs)
#plt.text(9,2100, r'75$^{th}$ %ile', **text_kwargs)


ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both', width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.grid(b=True, which='minor', linestyle='--')
#plt.xlabel(r'Algorithms')
plt.ylabel(r'Label')

plt.show
plt.savefig('boxplot_description2.png', transparent=True)